In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import optuna
from tqdm import tqdm

In [ ]:
#Defining E-model potential and derivative of potential

def f_Emod(Y, alpha):
    return (1 - np.exp(-np.sqrt(2/(3*alpha))*Y))**2
    
def df_Emod(Y, alpha):
    return 2*np.sqrt(2/(3*alpha))*(1 - np.exp(-np.sqrt(2/(3*alpha))*Y))*np.exp(-np.sqrt(2/(3*alpha))*Y)






#Defining T-model-1 potential and derivative of potential

def f_Tmod1(Y, alpha):
    return (np.tanh(Y/np.sqrt(6*alpha)))**2
    
def df_Tmod1(Y, alpha):
    return (2/np.sqrt(6*alpha))*np.tanh(Y/np.sqrt(6*alpha))/(np.cosh(Y/np.sqrt(6*alpha)))**2





#Defining T-model-2 potential and derivative of potential

def f_Tmod2(Y, alpha):
    return (np.tanh(Y/np.sqrt(6*alpha)))**4
    
def df_Tmod2(Y, alpha):
    return (4/np.sqrt(6*alpha))*((np.tanh(Y/np.sqrt(6*alpha)))**3)/((np.cosh(Y/np.sqrt(6*alpha)))**2)




#Defining function for the analytic approximations for T-model-1 and T-model-2 predictions

def analytic_approx(N, n, a):
    g = np.sqrt(3*a*(4*n**2 + 3*a))
    r = 12*a/(N**2 + N*g/(2*n) +3*a/4)
    ns = (1 - 2/N -3*a/(4*N**2) + g*(1-1/N)/(2*n*N))/(1 + g/(2*n*N) + 3*a/(4*N**2))
    return ns, r

In [ ]:
# This is the main prediction code, it takes as its inputs:

#                             alpha        :   parameter for the potential and its derivative
#                             k_piv        :   pivot scale we want to evaluate the predictions at
#                             rh_duration  :   duration of reheating in e-folds
#                             f            :   potential (without energy scale prefactor V0)
#                             df           :   derivative of potential (without energy scale prefactor V0)


def predict_ns_r(alpha, k_piv, rh_duration, f, df):

    V0 = 2.099e-9

    
    #Klein-Gordon equation as first-order system    
    def KG_system(t, y):
        return [y[1], -(3 - 0.5*y[1]**2)*y[1] - df(y[0],alpha)/((f(y[0], alpha)/(3 - 0.5*(y[1]**2))))]

    
    
    #Slow-roll approximation of dphi/dN    
    def dphidN(t, y):
        return -df(y, alpha)/f(y, alpha)
    
    
    
    #Function to get slow-roll initial conditions
    def get_SR_ICs(phi0):
        return [phi0, -df(phi0, alpha)/f(phi0, alpha)]
    
    
    
    #Initial search for field value where first Hubble slow-roll parameter is equal to or greater than 1
    phi_test_range = np.linspace(0.01, 20, 10000)
    ep_tests = 0.5*(df(phi_test_range, alpha)/f(phi_test_range, alpha))**2
    diff = np.abs(1 - ep_tests)
    phi_end_ind = diff.argmin()
    phi_end = phi_test_range[phi_end_ind]
    
    
    
    #Solving backwards to find initial field value
    N = np.linspace(0, -90, 10000)
    sol_init = solve_ivp(fun = dphidN,
                            t_span=[0,N[-1]],
                            t_eval = N,
                            y0= [phi_end]
                            )
    
    
    
    phi_i = sol_init.y[0][-1]
    
    
    
    #Full solution with slow-roll initial conditions and Radau method. This is what all subsequent
    #quantities are calculated from.
    N = np.linspace(0, 100, 10000)
    sol = solve_ivp(fun = KG_system,
                            t_span=[0,N[-1]],
                            t_eval = N,
                            y0= get_SR_ICs(phi_i),
                            method='Radau'
                            )
    

    phi = sol.y[0]
    phid = sol.y[1]
    
    
    #Computes the first Hubble slow-roll parameter
    epsilon = 0.5*phid**2
    
    
    
    #Finding the end of inflation and what we have defined as the end of slow roll in terms of epsilon
    ep_end = np.where(epsilon >= 1)[0][0] 
    ep_end_SR = np.where(epsilon >= 0.001)[0][0]

    
    
    #Computing the square of the Hubble parameter, field acceleration and second Hubble slow-roll parameter
    H2 = V0*f(phi, alpha)/(3 - 0.5*(phid**2))
    phidd = -(3 - 0.5*phid**2)*phid - df(phi, alpha)/(f(phi, alpha)/(3 - 0.5*(phid**2)))
    eta = 2*phidd/phid

    #Finding what we have defined as the end of slow roll in terms of the second Hubble slow-roll parameter 
    et_end_SR = np.where(np.abs(eta) >= 0.1)[0][0]
    
    
    #Computes the energy density of the field along with the time step where the different scales exit the horizon
    #using the initial guess for the energy scale V0. Also computes slow-roll scalar power spectrum.
    density = 0.5*H2*phid**2 + V0*f(phi, alpha)
    Ne = N[ep_end] - N[:ep_end]
    k = np.exp(67 - Ne + 0.25*np.log((V0*f(phi[:ep_end], alpha))**2) - 0.25*np.log(density[ep_end]) - np.log(2997.92) - np.log(1000)/12- rh_duration/4)*0.6736
    P_spec = H2[:ep_end]/(8*(np.pi**2)*epsilon[:ep_end])

    V0_error = []
    V0_trace = []
    
    
    #Finds the timestep where the scale closest to thepivot scale exits the horizon
    diff = np.abs(k - k_piv)
    k_piv_index = diff.argmin()
    Planck_As = 2.099e-9
    error = P_spec[k_piv_index]/Planck_As

    
    V0_trace.append(V0)
    V0_error.append(error)
    
    
    #Loop to iteratively adjust the V0 to match Planck_As at the pivot scale.
    while np.around(error, 3) != 1:
            
            if np.around(error, 3) > 1:
                V0 = V0*0.99

                H2 = V0*f(phi, alpha)/(3 - 0.5*(phid**2))
                density = 0.5*H2*(phid)**2 + V0*f(phi, alpha)
                k = np.exp(67 - Ne + 0.25*np.log((V0*f(phi[:ep_end], alpha))**2) - 0.25*np.log(density[ep_end]) - np.log(2997.92) - np.log(1000)/12- rh_duration/4)*0.6736
                P_spec = H2[ep_end]/(8*(np.pi**2)*epsilon[:ep_end])


                diff = np.abs(k - k_piv)
                k_piv_index = diff.argmin()
                error = P_spec[k_piv_index]/Planck_As
                
                
            elif np.around(error, 3) < 1:
                V0 = V0*1.01


                H2 = V0*f(phi, alpha)/(3 - 0.5*(phid**2))
                density = 0.5*H2*(phid)**2 + V0*f(phi, alpha)
                k = np.exp(67 - Ne + 0.25*np.log((V0*f(phi[:ep_end], alpha))**2) - 0.25*np.log(density[ep_end]) - np.log(2997.92) - np.log(1000)/12- rh_duration/4)*0.6736
                P_spec = H2[:ep_end]/(8*(np.pi**2)*epsilon[:ep_end])
                
        
                diff = np.abs(k - k_piv)
                k_piv_index = diff.argmin()
                error = P_spec[k_piv_index]/Planck_As

            V0_trace.append(V0)
            V0_error.append(error)


    #Calculates ns and r using first-order expressions
    ns = 1 - 2*epsilon[k_piv_index] - eta[k_piv_index]
    r = 16*epsilon[k_piv_index]

    return ns, r, Ne, k, P_spec, epsilon, eta, ep_end, k_piv_index, V0_trace, V0_error, phi, phid, phidd, H2



In [ ]:
alpha_trace=[]
rh_duration_trace=[]
loss_trace=[]
ns_trace=[]
r_trace=[]
targets=[]
best_alpha = []
bestLoss = []



def loss_function(ns_target, r_target, alpha, rh_duration):
    # Perform the calculations based on alpha and rh_duration
    ns, r, Ne, k, P_spec, epsilon, eta, ep_end, k_piv_index, V0_trace, V0_error, phi, phid, phidd, H2 = predict_ns_r(alpha, 0.05, rh_duration, f=f_Emod, df=df_Emod)
    ns_trace.append(ns)
    r_trace.append(r)
    # Calculate the mean squared error between the target values and the predictions
    mse_ns = 0.5*(ns - ns_target)**2
    mse_r = 0.5*(r - r_target)**2
    
    # Combine the MSEs into an overall loss
    loss = mse_ns + mse_r
    
    return loss



def objective(trial):
    # Define the search space for alpha and rh_duration
    alpha = trial.suggest_float('alpha', 0.01, 100)
    
    
    alpha_trace.append(alpha)
    rh_duration_trace.append(rh_duration)
    
    ns_target = 9.6526352E-01
    r_target = 1.6233650E-02
    
    # Calculate the loss using the defined loss function
    loss = loss_function(ns_target, r_target, alpha, rh_duration)
    loss_trace.append(loss)
    
    return loss



for duration in tqdm([0,15,30,40]):
    rh_duration = duration
    # Create a new Optuna study
    study = optuna.create_study(direction='minimize')

    # Run the optimization
    study.optimize(objective, n_trials=100)

    # Get the best parameter values and loss
    best_params = study.best_params
    best_loss = study.best_value

    best_alpha.append(best_params)
    bestLoss.append(best_loss)



In [ ]:
best_alpha_final = []

def objective(trial):

    upper_lim = best_alpha_1 + 2
    if best_alpha_1 <=2:
        lower_lim = 0.01
    else:
        lower_lim = best_alpha_1 -2
    


    alpha = trial.suggest_float('alpha', lower_lim, upper_lim)

    
    alpha_trace.append(alpha)
    rh_duration_trace.append(rh_duration)
    
    ns_target = 9.6526352E-01
    r_target = 1.6233650E-02
    
    
    loss = loss_function(ns_target, r_target, alpha, rh_duration)
    loss_trace.append(loss)
    
    return loss



sampler = optuna.samplers.TPESampler()


for i, duration in enumerate(tqdm([0,15,30,40])):
    rh_duration = duration
    best_alpha_1 = best_alpha[i]['alpha']
    # Create a new Optuna study
    study = optuna.create_study(direction='minimize', sampler=sampler)

    # Run the optimization
    study.optimize(objective, n_trials=100)

    # Get the best parameter values and loss
    best_params = study.best_params
    best_loss = study.best_value

    best_alpha_final.append(best_params)